In [1]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [5]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [9]:
for submission in reddit.subreddit("wallstreetbets").hot(limit=1):
    for comment in submission.comments:
        try:
            print(comment.body)
        except:
            pass

What are y'alls thoughts on these generic movie clips/memes being posted "the feeling when" or " xyz holders right now"??

Personally, I want to remove.  

But I totally respect this is just my opinion and want some community feedback, please.

Edit: 

##upvote stay, downvote remove 

(Easier than typing/reading)


^Double ^edit: ^Should've ^put ^a ^little ^more ^thought ^into ^karma ^voting ^direction.  ^Shit
All logic goes out the window with meme stocks. When you think its too expensive to buy it makes a new high the next day. Definitely gonna fomo into BB today
WHAT ARE YOU DOING STEP **BB**RO?
If BB hits 19$ today I will personally have a good nights rest and drink plenty of water.
Buying used BlackBerry’s hand over fist on eBay 🧠
Bb just getting started...
BB LETS BREAK 13 !!!!
just bought 10k bb let’s goooo
BB will open at 15
I have blue balls for BB
Starting to feel like BB is going back to ATH
300 shares into BB
Fomo'd into BB lol. Let's go
Omg BB is almost to where I bought i

In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    

In [ ]:
 els = soup.select('.jobsearch-SerpJobCard') 